In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

print('gen_user_question_feat...')

gen_user_question_feat...


In [3]:
data = pd.read_pickle(os.path.join(DATA_PATH, 'invite_data.pkl'))
user_info = pd.read_pickle(os.path.join(DATA_PATH, 'user_info.pkl'))
question_info = pd.read_pickle(os.path.join(DATA_PATH, 'question_info.pkl'))

In [4]:
invite_id = data[['uid', 'qid']]
invite_id_qm = invite_id.merge(user_info[['uid', 'topic_a', 'topic_i']], 'left', 'uid').merge(question_info[['qid', 'topic']], 'left', 'qid')

In [5]:
def gc_list(l):
    for i in l:
        del i
    del l
    gc.collect()

In [6]:
# 用户关注topic和问题 topic的交集
def process(df):
    return df.progress_apply(lambda row: list(set(row['topic_a']) & set(row['topic'])),axis=1)

with mp.Pool(8) as pool:
    ret = pool.map(process, np.array_split(invite_id_qm, 8))
invite_id_qm['topic_a_com'] = pd.concat(ret)
gc_list(ret)

100%|██████████| 1328855/1328855 [01:01<00:00, 21630.94it/s]


In [7]:
# 用户感兴趣topic和问题 topic的交集
def process(df):
    return df.progress_apply(lambda row: list(set(row['topic_i'].keys()) & set(row['topic'])),axis=1)

with mp.Pool(8) as pool:
    ret = pool.map(process, np.array_split(invite_id_qm, 8))
invite_id_qm['topic_i_com'] = pd.concat(ret)
gc_list(ret)

100%|██████████| 1328855/1328855 [00:59<00:00, 22352.09it/s]


In [8]:
def process(df):
    return df.progress_apply(lambda row: [row['topic_i'][t] for t in row['topic_i_com']],axis=1)

with mp.Pool(8) as pool:
    ret = pool.map(process, np.array_split(invite_id_qm, 8))
invite_id_qm['topic_iv_com'] = pd.concat(ret)
gc_list(ret)

100%|██████████| 1328855/1328855 [00:33<00:00, 39594.19it/s]


In [9]:
# 交集topic计数
invite_id_qm['num_topic_a_com'] = invite_id_qm['topic_a_com'].progress_apply(len)
invite_id_qm['num_topic_i_com'] = invite_id_qm['topic_i_com'].progress_apply(len)

100%|██████████| 10630845/10630845 [00:11<00:00, 926519.50it/s] 


In [10]:
# 交集topic兴趣值统计
invite_id_qm['topic_iv_com'] = invite_id_qm['topic_iv_com'].progress_apply(lambda x: [0] if len(x) == 0 else x)
invite_id_qm['min_topic_iv_com'] = invite_id_qm['topic_iv_com'].progress_apply(np.min)
invite_id_qm['max_topic_iv_com'] = invite_id_qm['topic_iv_com'].progress_apply(np.max)
invite_id_qm['mean_topic_iv_com'] = invite_id_qm['topic_iv_com'].progress_apply(np.mean)
invite_id_qm['std_topic_iv_com'] = invite_id_qm['topic_iv_com'].progress_apply(np.std)

100%|██████████| 10630845/10630845 [07:27<00:00, 23776.58it/s]


In [11]:
feats = ['num_topic_a_com', 'num_topic_i_com', 'min_topic_iv_com', 'max_topic_iv_com', 'mean_topic_iv_com', 'std_topic_iv_com']
feats += []
user_question_feat = invite_id_qm[feats]

In [12]:
user_question_feat.head(2)

,num_topic_a_com,num_topic_i_com,min_topic_iv_com,max_topic_iv_com,mean_topic_iv_com,std_topic_iv_com
0,1,0,0.0,0.0,0.0,0.0
1,0,0,0.0,0.0,0.0,0.0


In [13]:
user_question_feat.to_pickle(os.path.join(SAVE_PATH, 'user_question_feat.pkl'))

In [14]:
toc = time.time()
print('Used time: %d' % int(toc-tic))

Used time: 1200
